In [1]:
from typing import Union, Self
from numbers import Number
from itertools import chain, product

import numpy as np
import sympy as sp
import cvxpy as cp
import json
import hashlib
import scipy
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from scipy.linalg import qr
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import splu, svds
from sksparse.cholmod import cholesky
import xxhash
from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem, DoubleTraceOperator
from bmn.linear_algebra import get_null_space_dense, create_sparse_matrix_from_dict, get_row_space_dense, get_null_space_sparse
from bmn.bootstrap import BootstrapSystem
from bmn.solver import (
    minimal_eigval,
    sdp_init, sdp_relax,
    sdp_minimize,
    #minimize,
    get_quadratic_constraint_vector_sparse,
    get_quadratic_constraint_vector_dense,
    compute_L2_norm_of_quadratic_constraints,
)
from bmn.models import OneMatrix, TwoMatrix, MiniBFSS
from bmn.brezin import compute_Brezin_energy, compute_Brezin_energy_Han_conventions
import os
from bmn.debug_utils import disable_debug

import yaml
from bmn.config_utils import (
    run_bootstrap_from_config,
    generate_configs_one_matrix,
    generate_configs_two_matrix,
    generate_configs_three_matrix,
    generate_configs_bfss,
    )

from bmn.models import OneMatrix, TwoMatrix, MiniBFSS

# plot settings
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler
import torch
import torch.optim as optim
from torch.nn import ReLU

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5.0
plt.rcParams['xtick.minor.size'] = 3.0
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['lines.linewidth'] = 2
plt.rc('font', family='serif',size=16)
matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=16)
matplotlib.rcParams['axes.prop_cycle'] = cycler(
    color=['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8']
    )
matplotlib.rcParams.update(
    {"axes.grid":True,
    "grid.alpha":0.75,
    "grid.linewidth":0.5}
    )
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

np.set_printoptions(linewidth=120)  # Adjust the number to the desired width

In [2]:
model = MiniBFSS(couplings={"lambda": 1})
checkpoint_path = "checkpoints/MiniBFSS_L_4_lazy_symmetric"

#model = MiniBFSS(couplings={"lambda":1})
#checkpoint_path = "checkpoints/MiniBFSS_L_3_symmetric"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=4,
    symmetry_generators=model.symmetry_generators,
    verbose=True,
    checkpoint_path=checkpoint_path,
    odd_degree_vanish=True,
    #load_from_previously_computed=True
    )
bootstrap.load_constraints(checkpoint_path)

#17.7G

Assuming all operators are either Hermitian or anti-Hermitian.
NOTE Remember to incorporate more general basis changes!
Bootstrap system instantiated for 2015539 operators
Attribute: simplify_quadratic = True
Attempting to load from checkpoints, checkpoint_dir=checkpoints/MiniBFSS_L_4_lazy_symmetric
  loaded previously computed linear constraints
  loaded previously computed cyclic constraints


In [ ]:
len(bootstrap.linear_constraints)

In [3]:
new_constraints = []

set_of_hashes = set()
for st_operator in bootstrap.linear_constraints:
    new_dict = {}
    norm = 1
    for idx, (k, v) in enumerate(st_operator):
        if idx == 0:
            norm = float(1/v)
        new_dict[''.join(k)] = float(v) * norm
    #new_dict = {''.join(k): float(v) for k, v in st_operator}
    s = json.dumps(new_dict).encode('utf-8')
    hash_digest = hashlib.md5(s).digest()
    if hash_digest not in set_of_hashes:
        new_constraints.append(st_operator)
        set_of_hashes.add(hash_digest)

In [4]:
print(f"number of unique hashes: {len(set_of_hashes)}, ratio of hashes to constraints: {len(set_of_hashes) / len(bootstrap.linear_constraints)}")

number of unique hashes: 5613398, ratio of hashes to constraints: 0.4418032922361312


In [5]:
bootstrap.linear_constraints = new_constraints

In [6]:
linear_constraint_matrix = bootstrap.build_linear_constraints()

In [7]:
linear_constraint_matrix.shape

(5613398, 2015539)

In [8]:
bootstrap.build_null_space_matrix()

DEBUG 22:23:58.559853: Building the null space matrix. The linear constraint matrix has dimensions (5613398, 2015539)


: 

: 

In [ ]:
model = TwoMatrix(couplings={"g2":0, "g4": 1})
checkpoint_path = "checkpoints/TwoMatrix_L_4_symmetric_energy_fixed_g2_0"

#model = MiniBFSS(couplings={"lambda":1})
#checkpoint_path = "checkpoints/MiniBFSS_L_3_symmetric"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=4,
    symmetry_generators=model.symmetry_generators,
    verbose=True,
    checkpoint_path=checkpoint_path,
    odd_degree_vanish=True,
    )
bootstrap.load_constraints(checkpoint_path)

In [ ]:
if bootstrap.linear_constraints is None:
    _ = bootstrap.build_linear_constraints().tocsr()

# quadratic constraints
if bootstrap.quadratic_constraints_numerical is None:
    bootstrap.build_quadratic_constraints()
quadratic_constraints_numerical = bootstrap.quadratic_constraints_numerical

# bootstrap table
if bootstrap.bootstrap_table_sparse is None:
    bootstrap.build_bootstrap_table()
bootstrap_table_sparse = bootstrap.bootstrap_table_sparse

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
# build the Ax = b constraint
Avec = bootstrap.single_trace_to_coefficient_vector(
    SingleTraceOperator(data={(): 1}), return_null_basis=True
)
Avec = torch.from_numpy(Avec).type(torch.float).to(device)

# Hamiltonian vector
Hvec = bootstrap.single_trace_to_coefficient_vector(
    bootstrap.hamiltonian, return_null_basis=True
)
Hvec = torch.from_numpy(Hvec).type(torch.float).to(device)

# build the bootstrap array
bootstrap_array_sparse = bootstrap.bootstrap_table_sparse
bootstrap_array_torch = torch.from_numpy(bootstrap_array_sparse.todense()).type(torch.float).to(device)

# build the constraints
quadratic_constraints = bootstrap.quadratic_constraints_numerical
quadratic_constraint_linear = torch.from_numpy(quadratic_constraints["linear"].todense()).type(torch.float).to(device)
quadratic_constraint_quadratic = torch.from_numpy(quadratic_constraints["quadratic"].todense()).type(torch.float).to(device)
quadratic_constraint_quadratic = quadratic_constraint_quadratic.reshape((len(quadratic_constraint_quadratic), len(Hvec), len(Hvec)))

In [ ]:
def energy(param):
    return Hvec @ param


def get_quadratic_constraint_vector(param):
    quadratic_constraints = torch.einsum(
        "Iab, a, b -> I", quadratic_constraint_quadratic, param, param
    ) + torch.einsum("Ia, a -> I", quadratic_constraint_linear, param)
    return torch.square(quadratic_constraints)


def quadratic_loss(param):
    return torch.sum(get_quadratic_constraint_vector(param))


def Axb_loss(param):
    return torch.square(Avec @ param - 1)


def psd_loss(param):
    bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )  # is this reshaping correct?
    smallest_eigv = torch.linalg.eigvalsh(bootstrap_matrix)[0]
    return ReLU()(-smallest_eigv)


def build_loss(param):
    lambda_psd = 1e2
    lambda_quadratic = 1e2
    lambda_Axb = 1e2
    loss = (
        energy(param)
        + lambda_psd * psd_loss(param)
        + lambda_quadratic * quadratic_loss(param)
        + lambda_Axb * Axb_loss(param)
    )
    return loss

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR

In [ ]:
## Consider putting this on the GPU

# Training loop
#param = 1e0 * torch.randn(bootstrap.param_dim_null).to(device)
param = torch.tensor(np.asarray([1/Avec[0].item()] + [0] * (len(Hvec)-1))).type(torch.float).to(device)
param.requires_grad = True

optimizer = optim.Adam([param], lr=1e-1)
scheduler = ExponentialLR(optimizer, gamma=1-1e-3)
num_epochs = 100_000

for epoch in range(num_epochs):
    optimizer.zero_grad()
    loss = build_loss(param)
    loss.backward()
    optimizer.step()
    scheduler.step()

    if (epoch + 1) % 100 == 0:
        print(f"epoch {epoch+1}/{num_epochs}, loss: {loss.item()}, lr: {scheduler.get_lr()[0]:.4e}")

In [ ]:
# Questions:
# 1. why is it sparse?
# 2. why is it so close to the L=3 case?
param

In [ ]:
energy(param)

In [ ]:
psd_loss(param), quadratic_loss(param), Axb_loss(param)

In [ ]:
def energy(param):
    return Hvec @ param


def get_quadratic_constraint_vector(param):
    quadratic_constraints = torch.einsum(
        "Iab, a, b -> I", quadratic_constraint_quadratic, param, param
    ) + torch.einsum("Ia, a -> I", quadratic_constraint_linear, param)
    return torch.square(quadratic_constraints)


def quadratic_loss(param):
    return torch.sum(get_quadratic_constraint_vector(param))


def Axb_loss(param):
    return torch.square(Avec @ param - 1)


def psd_loss(param):
    bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )  # is this reshaping correct?
    smallest_eigv = torch.linalg.eigvalsh(bootstrap_matrix)[0]
    return ReLU()(-smallest_eigv)


def build_loss(param):
    lambda_psd = 1e2
    lambda_quadratic = 1e2
    lambda_Axb = 1e2
    loss = (
        energy(param)
        + lambda_psd * psd_loss(param)
        + lambda_quadratic * quadratic_loss(param)
        + lambda_Axb * Axb_loss(param)
    )
    return loss

In [ ]:
# Training loop
param = 1e0 * torch.randn(bootstrap.param_dim_null)
param.requires_grad = True

optimizer = optim.Adam([param], lr=1e-3)
num_epochs = 100_000

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Clear previous gradients

    loss = build_loss(param)  # Compute the loss
    loss.backward()  # Compute gradients
    optimizer.step()  # Update the parameters

    # Print the loss for monitoring
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
energy(param)

In [ ]:
psd_loss(param), quadratic_loss(param), Axb_loss(param)

In [ ]:
np.where(np.abs(Hvec.numpy()) > 1e-10)[0]

In [ ]:
i = 8
param[i]

In [ ]:
get_quadratic_constraint_vector(param)

In [ ]:
84**2

In [ ]:
bootstrap_array_sparse

In [ ]:
param

In [ ]:
bootstrap.single_trace_to_coefficient_vector(model.operators_to_track["x_2"], return_null_basis=True)

In [ ]:
bootstrap.single_trace_to_coefficient_vector(model.operators_to_track["p_2"], return_null_basis=True)

In [ ]:
bootstrap.single_trace_to_coefficient_vector(model.operators_to_track["x_4"], return_null_basis=True)

In [ ]:
bootstrap_array_torch.shape

In [ ]:
print(list((bootstrap_array_torch @ param).reshape((bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)).detach().numpy()))

In [ ]:
type(set())

In [ ]:
from bmn.models import TwoMatrix, MiniBFSS

#model = TwoMatrix(couplings={"g2": 0, "g4": 1})
#checkpoint_path = "checkpoints/TwoMatrix_L_3_symmetric_energy_fixed_g2_0"

model = MiniBFSS(couplings={"lambda": 1})
checkpoint_path = "checkpoints/MiniBFSS_L_3_symmetric"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=3,
    symmetry_generators=model.symmetry_generators,
    verbose=True,
    checkpoint_path=checkpoint_path
    )
bootstrap.load_constraints(checkpoint_path)

In [ ]:
st_operator_inhomo_constraints=[
    (SingleTraceOperator(data={(): 1}), 1)
    ]

A = sparse.csr_matrix((0, bootstrap.param_dim_null))
b = np.zeros(0)
for op, value in st_operator_inhomo_constraints:

    A = sparse.vstack(
        [
            A,
            sparse.csr_matrix(
                bootstrap.single_trace_to_coefficient_vector(
                    op, return_null_basis=True
                )
            ),
        ]
    )
    b = np.append(b, value)
linear_inhomogeneous_eq_no_quadratic = (A, b)

In [ ]:
A.todense()

In [ ]:
bootstrap.null_space_matrix.shape

In [ ]:
bootstrap.null_space_matrix[0].todense()

In [ ]:
commutator = model.matrix_system.single_trace_commutator(
    st_operator1=model.hamiltonian,
    st_operator2=SingleTraceOperator(data={("X0", "Pi0"):1, ("X1", "Pi1"):1}),
)

commutator = model.matrix_system.single_trace_commutator(
    st_operator1=SingleTraceOperator(data={("X0", "Pi0"):1, ("X1", "Pi1"):1}),
    st_operator2=model.hamiltonian,
)

commutator_me = SingleTraceOperator(
    data={
    ("Pi0", "Pi0"): -2 / 2,
    ("Pi1", "Pi1"): -2 / 2,
    ("X0", "X0"): -2 * model.couplings["g2"] / 2,
    ("X1", "X1"): -2 * model.couplings["g2"] / 2,
    ("X0", "X1", "X0", "X1"): 4*model.couplings["g4"] / 4,
    ("X1", "X0", "X1", "X0"): 4*model.couplings["g4"] / 4,
    ("X0", "X1", "X1", "X0"): -4*model.couplings["g4"] / 4,
    ("X1", "X0", "X0", "X1"): -4*model.couplings["g4"] / 4,
    }
    )
commutator == -commutator_me

TODO
- consolidate 1, 2, 3, and BFSS models
- consolidate generate_model_X_config 
- move checkpoints, maybe add info on the checkpoints

In [ ]:
generate_configs_bfss(
    config_filename=f"test",
    config_dir="MiniBFSS_L_3_test",
    st_operator_to_minimize="x_2",
    st_operators_evs_to_set={"energy": 1},
    load_from_previously_computed=True,
    impose_symmetries=True,
    reg_decay_rate=0.7880462815669912,
    )

# python bmn/config_utils.py run_bootstrap_from_config -config_filename test -config_dir MiniBFSS_L_3_test

In [ ]:
reg_decay_rate = np.exp(-3 * np.log(10) / (30 - 1))
reg_decay_rate**(30-1), reg_decay_rate

In [ ]:
generate_configs_bfss(
    config_filename=f"test",
    config_dir="MiniBFSS_L_3_test",
    st_operators_evs_to_set={"energy": 1},
    load_from_previously_computed=True,
    impose_symmetries=True,
    tol=1e-2,
    maxiters=50,
    maxiters_cvxpy=10_000,
    init_scale=1e2,
    penalty_reg=1e6,
    reg=1e-4,
    #reg_decay_rate=reg_decay_rate,
    )

# python bmn/config_utils.py run_bootstrap_from_config -config_filename test_2 -config_dir MiniBFSS_L_3_test

In [ ]:
1.9525 / 3

In [ ]:
generate_configs_two_matrix(
    config_filename=f"test_small_mass_hold_energy_fixed",
    config_dir="TwoMatrix_L_3_symmetric",
    st_operator_to_minimize="x_2",
    st_operators_evs_to_set={"energy": 1},
    g2=1e-4,
    g4=1,
    load_from_previously_computed=True,
    impose_symmetries=True,
    )

In [ ]:
generate_configs_three_matrix(
    config_filename=f"test_small_mass",
    config_dir="ThreeMatrix_L_3_symmetric",
    g2=1e-4,
    g4=1,
    load_from_previously_computed=False,
    impose_symmetries=True,
    )

In [ ]:
run_bootstrap_from_config(
    config_filename=f"g4_{str(0.448)}",
    config_dir="TwoMatrix_L_4_symmetric",
    verbose=False,
    )

In [ ]:
for reg in [1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9]:

    generate_configs_bfss(
        config_filename=f"reg_{int(reg)}",
        config_dir="MiniBFSS_L_3_symmetric",
        energy=1,
        st_operator_to_minimize="x_2",
        load_from_previously_computed=True,
        impose_symmetries=True,
        maxiters_cvxpy=5_000,
        maxiters=100,
        radius=1e6,
        reg=reg,
        )

In [ ]:
st_operator_to_minimize = "x_2"

generate_configs_bfss(
    "MiniBFSS_symmetric_L_3",
    energy=1,
    st_operator_to_minimize="x_2",
    load_from_previously_computed=True,
    impose_symmetries=True,
    maxiters_cvxpy=10_000,
    maxiters=100,
    radius=1e8,
    )

generate_configs_bfss(
    "MiniBFSS_L_3",
    energy=1,
    st_operator_to_minimize="x_2",
    load_from_previously_computed=True,
    impose_symmetries=False,
    maxiters_cvxpy=10_000,
    maxiters=100,
    radius=1e8,
    )

#run_bootstrap_from_config("hello", verbose=False)

## BFSS

In [ ]:
L = 2

matrix_system = MatrixSystem(
    operator_basis=["X0", "X1", "X2", "Pi0", "Pi1", "Pi2"],
    commutation_rules_concise={
        ("Pi0", "X0"): 1,  # use Pi' = i P to ensure reality
        ("Pi1", "X1"): 1,
        ("Pi2", "X2"): 1,
    },
    hermitian_dict={
        "Pi0": False,
        "X0": True,
        "Pi1": False,
        "X1": True,
        "Pi2": False,
        "X2": True,
        },
)

# lambda = 1 here
hamiltonian = SingleTraceOperator(
    data={
        ("Pi0", "Pi0"): -0.5,
        ("Pi1", "Pi1"): -0.5,
        ("Pi2", "Pi2"): -0.5,
        # quartic term (XY)
        ("X0", "X1", "X0", "X1"): -1 / 4,
        ("X1", "X0", "X1", "X0"): -1 / 4,
        ("X0", "X1", "X1", "X0"): 1 / 4,
        ("X1", "X0", "X0", "X1"): 1 / 4,
        # quartic term (XZ)
        ("X0", "X2", "X0", "X2"): -1 / 4,
        ("X2", "X0", "X2", "X0"): -1 / 4,
        ("X0", "X2", "X2", "X0"): 1 / 4,
        ("X2", "X0", "X0", "X2"): 1 / 4,
        # quartic term (YZ)
        ("X1", "X2", "X1", "X2"): -1 / 4,
        ("X2", "X1", "X2", "X1"): -1 / 4,
        ("X1", "X2", "X2", "X1"): 1 / 4,
        ("X2", "X1", "X1", "X2"): 1 / 4,
    }
)

# <tr G O > = 0
gauge = MatrixOperator(data={
    ("X0", "Pi0"): 1,
    ("Pi0", "X0"): -1,
    ("X1", "Pi1"): 1,
    ("Pi1", "X1"): -1,
    ("X2", "Pi2"): 1,
    ("Pi2", "X2"): -1,
    (): 3,
    })

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge_generator=gauge,
    max_degree_L=L,
    odd_degree_vanish=True,
    simplify_quadratic=True,
)

bootstrap.build_quadratic_constraints()

quadratic_constraints_numerical = bootstrap.quadratic_constraints_numerical

In [ ]:
quadratic_constraints_numerical

In [ ]:
def newton_pseudoinverse(param, nsteps, quadratic_constraints_numerical):

    for step in range(nsteps):
        val, grad = get_quadratic_constraint_vector_sparse(
            quadratic_constraints=quadratic_constraints_numerical,
            param=param,
            compute_grad=True,
            )
        grad_pinv = np.linalg.pinv(grad)
        #np.allclose(grad @ grad_pinv @ grad, grad)
        #np.allclose(grad.T @ np.linalg.inv(grad @ grad.T), grad_pinv)
        param = param - np.asarray(grad_pinv @ val)[0]
        print(f"Newton's method: step = {step}, val = {np.linalg.norm(val)}")

    return param

In [ ]:
bootstrap.param_dim_null

In [ ]:
167958/3

In [ ]:
param = np.random.normal(size=bootstrap.param_dim_null)
val = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    )

param = newton_pseudoinverse(param=param, nsteps=10, quadratic_constraints_numerical=quadratic_constraints_numerical)
val = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    )

In [ ]:
val.shape, grad.T.shape

In [ ]:
val.shape, grad.shape, grad_pinv.shape

In [ ]:
val2, grad2 = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

In [ ]:
np.sum(np.abs(grad2[0]))

In [ ]:
grad2.shape

In [ ]:
min(np.sum(np.abs(np.asarray(grad2)), axis=1)), max(np.sum(np.abs(np.asarray(grad2)), axis=1))

In [ ]:
min(np.abs(val2)), max(np.abs(val2))

In [ ]:
val1, grad1 = get_quadratic_constraint_vector_dense(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

val2, grad2 = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

In [ ]:
np.allclose(val1, val2), np.allclose(grad1, grad2)

## BMN

In [ ]:
from bmn.models import MiniBMN
from bmn.bootstrap_complex import BootstrapSystemComplex

model = MiniBMN(couplings={"g2": 1, "g4": 1})

bootstrap = BootstrapSystemComplex(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=3,
    simplify_quadratic=False,
    symmetry_generators=None,
    fraction_operators_to_retain=0.41,
    )

In [ ]:
bootstrap.build_null_space_matrix()

In [ ]:
bootstrap.build_linear_constraints()

In [ ]:
bootstrap.build_quadratic_constraints()

In [ ]:
bootstrap.quadratic_constraints_numerical["quadratic"][0]

In [ ]:
bootstrap.build_bootstrap_table()

In [ ]:
from bmn.newton_solver import solve_bootstrap

param, optimization_result = solve_bootstrap(
    bootstrap=bootstrap,
    st_operator_to_minimize=bootstrap.hamiltonian,
    )

In [ ]:
x = 1 + 1j
x.real

In [ ]:
x = 1
x.real